# Record entire power runs
Goal is to screen cap and also merge the webcam feed looking at power display

WHEN RECORDING: fill the terminal to ensure print appear in the right place 

WHEN RECORDING: put the terminal on the bottom right corner of the vertical screen

In [10]:
import cv2
import numpy as np
from mss import mss
from os import listdir
from os.path import isfile, join


import cupy as cnp
import csv
import pytesseract
import easyocr
from IPython.display import clear_output

reader = easyocr.Reader(
    ["en"]
)  # this needs to run only once to load the model into memory

def clean_square_for_OCR(image):
    grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(grey, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 50:
            cv2.drawContours(opening, [c], -1, 0, -1)

    result = 255 - opening
    result = opening
    result = cv2.GaussianBlur(result, (5, 5), 0)
    return result


def fix(screen):
    screen = screen.swapaxes(0, 2)
    screen = screen[:][:][:3]
    screen = screen.swapaxes(0, 2)
    return screen


# write code to create new image each add two images together increasing the size of the merged image
def merge(screen, i2, x_offset, y_offset):
    screen[y_offset : y_offset + i2.shape[0], x_offset : x_offset + i2.shape[1]] = i2
    return screen


# Output file
name = "test"  # set name
mypath = "./files/"
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
outname = mypath + f"{name}.mp4"

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(outname, fourcc, 20.0, (1280, 960))

# Capture from camera and screen
cap = cv2.VideoCapture(-1)
sct = mss()
bounding_box = {"top": 2000, "left": 0, "width": 1920, "height": 1740}
if not cap.isOpened():
    print("Cannot open camera")
    exit()

#load coords
try:
    with open(mypath + "coords.txt", "r") as f:
        px, py, pxs, pys = [int(x) for x in f.read().split()]
except:
    print("No coords.txt found")
    px, py, pxs, pys = (82, 298, 350, 80)
# px, py, pxs, pys = (82, 298, 350, 80)
target_mode = False
while True:
    # Screen Capture
    screen = np.array(sct.grab(bounding_box))
    scale_percent = 50  # percent of original size
    width = int(screen.shape[1] * scale_percent / 100)
    height = int(screen.shape[0] * scale_percent / 100)
    dim = (width, height)
    screen = cv2.resize(screen, dim, interpolation=cv2.INTER_AREA)
    screen = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
    screen_S = screen.shape
    screen = np.concatenate(
        (screen, np.zeros((screen.shape[0], 200, 3), dtype=np.uint8)), axis=1
    )

    # Camera Capture
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.ROTATE_180)

    key = cv2.waitKey(1)
    if key == ord("w"):
        py -= 1
    if key == ord("s"):
        py += 1

    if key== ord("a"):
        px -= 1
    if key == ord("d"):
        px += 1
    

    frame = frame[py : py + pys, px : px + pxs, :3]  # // adjust to focus on mWH
    # rescale down
    # frame = cv2.resize(frame, (int(pxs/2), int(pys/2)), interpolation = cv2.INTER_AREA)

    scale_percent = 200  # percent of original size
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    # frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


    # Merge Screen and Camera
    frame = merge(screen, frame, screen_S[0] - 60, 20)

    # Write the frame
    frame = cv2.resize(frame, (1280, 960))
    out.write(frame)

    if target_mode:
        power = frame[20:100, 920:-135, :]  # y , x // adjust to focus on mWH
        power = cv2.resize(power, (power.shape[1] * 4, power.shape[0] * 4))
        npower = clean_square_for_OCR(power)
        power_ocr = pytesseract.image_to_string(
            npower, lang="eng", config="--psm 8 -c tessedit_char_whitelist=0123456789"
        ).replace("\n\x0c", "")
        print(power_ocr)
        cv2.imshow("npower", npower)
        if key == ord("q"):
            break
        elif key == ord("c"):
            target_mode = False
            cv2.destroyAllWindows()
            clear_output(wait=True)
        continue



    # Display the resulting frame
    cv2.imshow("frame", frame)
    if key== ord("q"):
        break
    elif key == ord("c"):
        target_mode = True
        cv2.destroyAllWindows()
        clear_output(wait=True)

print( f"({px}, {py}, {pxs}, {pys})")
# save to coord to coords.txt
with open(mypath + "coords.txt", "w") as f:
    f.write(f"{px} {py} {pxs} {pys}")
# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()

200593
200593
200593
200593
200593
200593
200593
200594
200594
(43, 363, 350, 80)
